# Fidelity calculation using QuTIP

In [97]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import qcd
from qcd.configurations.configuration import ChannelConfiguration
from qcd.configurations import OneShotConfiguration, OneShotEntangledConfiguration
import numpy as np
import math
from typing import Optional, Tuple, cast, List, Dict
from qutip import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.quantum_info.states.utils import partial_trace
from qiskit.result.result import Result
import pickle

## Variables to control

In [98]:
# Filenames with results for 3 etas and their optimized parameters to LOAD
# filename = '20210416a_C2_A2_500_10000'  
filename = '20210414a_C2_A2_100_100_3'
#filename = '20210413a_C2_A3_500_10000'  # 2 RX + 2 RY gates
n_channels = 2

## Auxiliary functions

In [99]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name, path):
    with open(path + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [100]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)


In [101]:
def _run_all_circuits_and_return_density_matrices(configuration: ChannelConfiguration):
    """ Create a pair of Quantum Circuits, in its transpiled form, from a given configuration """

    state_vectors = [cast(Result, execute(_create_one_circuit(configuration, eta),
                                         backend=GLOBAL_BACKEND).result()).get_statevector()
                    for eta in configuration.eta_group]

    # print(state_vectors)
    d_m = [[], []] 
    d_m[0] = partial_trace(np.outer(state_vectors[0],state_vectors[0]), [2])  
    d_m[1] = partial_trace(np.outer(state_vectors[1],state_vectors[1]), [2])  
    # print(type(d_m[0]))
    return d_m


In [102]:
#    print(configuration['eta_pair'])
#    print(state_vectors)
#    d_m = [[], []] 
#    for idx, eta in enumerate(configuration['eta_pair']):
        #print('idx = ' + str(idx))
#        d_m[idx] = partial_trace(np.outer(state_vectors[idx],state_vectors[idx]), [2])
#        print(QObj(d_m[idx]))
#        print(d_m)
#    return d_m

In [103]:
def _create_one_circuit(configuration: ChannelConfiguration,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given  configuration and eta """
    configuration = cast(OneShotEntangledConfiguration, configuration)
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration.state_probability)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.rx(configuration.angle_rx1, qreg_q[1])
    circuit.ry(configuration.angle_ry1, qreg_q[1])
    circuit.rx(configuration.angle_rx0, qreg_q[0])
    circuit.ry(configuration.angle_ry0, qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

## Fidelity calculations
At this point only pairs of etas are considered.


In [104]:
trio_results = load_results_from_file(name=filename, path='./../demo/data/')

In [132]:
GLOBAL_BACKEND=Aer.get_backend('statevector_simulator')
GLOBAL_ETA_TRIOS = trio_results['eta_groups']
POVM_configurations = trio_results['configurations']
#density_matrices = [[], []]
fidelity_list = []
fidelity_results = trio_results
for idx, POVM_configuration in enumerate(POVM_configurations):
    density_matrices = _run_all_circuits_and_return_density_matrices(POVM_configuration)
    fidelity_list.append(fidelity(Qobj(density_matrices[0].data), Qobj(density_matrices[1].data)))
    # print(density_matrices[0])
    # print(density_matrices[1])
    # print(density_matrices[0].to_dict())
    # print(density_matrices[1].to_dict())

    # print(Qobj(density_matrices[0]))
    # print(Qobj(density_matrices[1]))
    
    #test = ket2dm(Qobj(density_matrices[0]))
    #print(test.ptrace(2))
    # print(fidelity(Qobj(density_matrices[0]), Qobj(density_matrices[1])))


In [136]:
fidelity_list

[0.0,
 0.9996338760499143,
 0.7997595296008693,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.8898783081198488,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9544880676558882,
 0.0,
 0.0,
 0.7311953462987989,
 0.0,
 0.0,
 0.990481969449541,
 0.0,
 0.9081821707977438,
 0.0,
 0.9269673735718478,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.7524609169654681,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9999999999999999,
 0.0,
 0.9349201457280144,
 0.9999999999999997,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.9999999999999999,
 0.0,
 0.0,
 0.9999999999999999,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8581680909622179,
 0.9380828524074797,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9999999999999999,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9029429020566233,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9716159558029362,
 0

In [129]:
density_matrices[0].data

array([[0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.-8.34403085e-16j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j,
        0.+0.00000000e+00j]])

In [124]:
qubit = np.array([[0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 1.-8.34403085e-16j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j]])

In [125]:
qubit2 = np.array([[1.-2.87008388e-16j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j],
               [0.+0.00000000e+00j, 0.+0.00000000e+00j,
                0.+0.00000000e+00j, 0.+0.00000000e+00j]])

In [131]:
fidelity(Qobj(density_matrices[0].data), Qobj(density_matrices[1].data))

0.0

In [85]:
one_value = list(density_matrices[0].to_dict().values())[0]

In [117]:
Qobj(qubit)

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1.-2.86724386e-16j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]]

In [86]:
one_value

(1-3.90432278185857e-18j)

In [109]:
ptrace(ket2dm(basis(4,4)))

ValueError: All basis indices must be `offset <= n < dimension+offset`.

In [111]:
ket2dm(basis(4,4))

ValueError: All basis indices must be `offset <= n < dimension+offset`.

In [94]:
Qobj()

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[1.-3.90432278e-18j]]

In [53]:
x=basis(3,2)


In [55]:
ket2dm(x)

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

In [92]:
state_number_qobj([2, 1], [1, 0]) 

Quantum object: dims = [[2, 1], [1, 1]], shape = (2, 1), type = ket
Qobj data =
[[0.]
 [1.]]

In [89]:
ket('10')

TypeError: object of type 'numpy.complex128' has no len()